Pipeline for Extracting Text from Aventura Joven Books

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=da1f07d9bf0dc80df2a9e94106ff13519ac5af6cd7942f26247dee29291af2dc
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
#file_path = "/content/drive/MyDrive/capstone/Aventura Joven 03 - Perdidos en - Elvira Sancho, Jordi Suris.pdf"
file

In [3]:
# import parser object from tike
from tika import parser  
import unicodedata
  
# opening pdf file
parsed_pdf = parser.from_file(file_path)
  
# parsed_pdf['content'] returns string 
data = parsed_pdf['content'] 

2021-05-12 14:54:19,052 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-05-12 14:54:27,972 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-05-12 14:54:29,749 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2021-05-12 14:54:34,761 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [8]:
# Find beginning of text
beg_idx = data.index('CAPÍtULO 1')
# Find end of text
end_idx = data.index('DESPUÉS')

print(beg_idx)
print(end_idx)

1886
56387


In [ ]:
# Check that this is indeed only the story text
text = data[beg_idx:end_idx]
print(data[beg_idx:end_idx])

In [10]:
# write the initial text to a file (no preprocessing done here)
with open('/content/drive/MyDrive/capstone/Aventura Joven 03 Text No Preproc.txt', 'w') as f:
  f.write(text)

Preprocessing the text

In [ ]:
# remove words with numbers in them, ex altura1 or hostal2 (footnote indicator)

import re

# remove special characters for regex
norm_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')

text_wo_ft_words = norm_text
for word in re.findall('[A-z]+[1234567890]{1,2}', text_wo_ft_words):
  word_idx = text_wo_ft_words.index(word)
  m = re.search('[A-z]+', word)
  stripped_word = m.group(0)
  text_wo_ft_words = text_wo_ft_words[:word_idx] + stripped_word + text_wo_ft_words[word_idx+len(word):]
            
# print(text_wo_ft_words[:2000])

In [17]:
with open('/content/drive/MyDrive/capstone/Aventura Joven 03 Text No Footnoted Words.txt', 'w') as f:
  f.write(text_wo_ft_words)

In [ ]:
# remove footnote numbers on their own in the text

In [22]:
print(text_wo_ft_words[200:600])
text_wo_ft_words[200:600]

i. Mis padres me han hablado mucho del Camino del Inca... 
Y tambien quiero visitar Cusco, a la vuelta. Guille, te encuentras 
bien? Estas un poco blanco...

Si, estoy un poco mareado, es por la altura.
Guille y Monica son amigos. El chico es un poco gordo y pelirro-

jo. Tiene dieciseis anos pero parece mas joven. 
Hola! Laura, una chica alta y rubia, entra en la habita-

cion. Estais listos?
Si 


'i. Mis padres me han hablado mucho del Camino del Inca... \nY tambien quiero visitar Cusco, a la vuelta. Guille, te encuentras \nbien? Estas un poco blanco...\n\nSi, estoy un poco mareado, es por la altura.\nGuille y Monica son amigos. El chico es un poco gordo y pelirro-\n\njo. Tiene dieciseis anos pero parece mas joven. \nHola! Laura, una chica alta y rubia, entra en la habita-\n\ncion. Estais listos?\nSi '

In [ ]:
def modify_words(regex_find, regex_modify, text, modify):
  ''' Modifies all words in text that match the regex_find regex to match the regex_modify regex. '''
  text_removed = text
  for word in re.findall(regex_find, text_removed):
    word_idx = text_removed.index(word)
    
    if modify == 'regex':
      m = re.search(regex_modify, word)
      modified_word = m.group(0)
    elif modify == 'remove':
      rm_char_idx = word.index(regex_modify)
      modified_word = word[:rm_char_idx] + word[rm_char_idx+len(regex_modify):]
    
    text_removed = text_removed[:word_idx] + modified_word + text_removed[word_idx+len(word):]

  return text_removed

In [35]:
# remove words that contain '-\n' because they didn't fit on one line
text_wo_broken_words = text_wo_ft_words
for word in re.findall('[A-z]+\-\n{1,2}[a-z]+', text_wo_broken_words):
  print(word)
  print('######')
  word_idx = text_wo_broken_words.index(word)
  hyphen_idx = word.index('-')
  if '-\n\n' in word:
    len_sep = 3
  else:
    len_sep = 2
  modified_word = word[:hyphen_idx] + word[hyphen_idx+len_sep:]
  print(modified_word)
  print('-------------------------------')
  print(text_wo_broken_words[word_idx-50:word_idx] + modified_word + text_wo_broken_words[word_idx+len(word):word_idx+len(word)+50])
  text_wo_broken_words = text_wo_broken_words[:word_idx] + modified_word + text_wo_broken_words[word_idx+len(word):]

print(text_wo_broken_words[:1000])

entran-
do
######
entrando
-------------------------------
uille, tu llevas camara de fotos? pregunta Monica entrando en la habitacion de los chicos.

No, pero Sergio 
pelirro-

jo
######
pelirrojo
-------------------------------
 y Monica son amigos. El chico es un poco gordo y pelirrojo. Tiene dieciseis anos pero parece mas joven. 
Hol
habita-

cion
######
habitacion
-------------------------------

Hola! Laura, una chica alta y rubia, entra en la habitacion. Estais listos?
Si Guille se pone en la espalda s
com-
batirse
######
combatirse
-------------------------------
mareos, trastornos estomacales y cansancio. Puede combatirse con pastillas, ejercicios de respiracion o mate.

can-

timplora
######
cantimplora
-------------------------------
eneis todo?, camaras?, linternas?, crema de sol?, cantimplora? pregunta Nuria.
Los alumnos de la clase de cuart
profe-
sores
######
profesores
-------------------------------
lona, estan en Peru de viaje de fin de curso. Dos profesores acompanan

In [ ]:
text_wo_broken_words = modify_words('[A-z]+\-\n[a-z]+', '-\n', text_wo_ft_words, 'remove')
print(text_wo_broken_words[:1000])

In [ ]:
# remove the footnotes

In [ ]:
# Attempt at removing the footnotes
import re

# normalize the data to ignore special characters in Spanish
norm_data = unicodedata.normalize('NFD', data).encode('ascii', 'ignore').decode('utf-8')
norm_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')

m = re.search('[A-z]+[1234567890]{1,2}', norm_data[1880:])

m.group(0)
footnoted_words = [word.strip() for word in re.findall('[A-z]+[1234567890]', norm_data[1880:])]
footnote_defs = [word.strip() for word in re.findall('[0-9]{1,2}  [A-z]+[\s\w]*:[\w \n]*\.', norm_data[1880:])]

#print(footnoted_words)
#print(footnote_defs)
print(re.findall('([0-9]{1,2}  [A-z]+[\s\w]*:([\w ,\.:\-]+\n)*\n)', '''1  altura: Cusco esta a unos 3500 metros de altura, lo que en algunas personas 
provoca el llamado mal de altura o soroche. Los sintomas del mal de altura 
son: dolor de cabeza, mareos, trastornos estomacales y cansancio. Puede com-
batirse con pastillas, ejercicios de respiracion o mate.

2  hostal: alojamiento normalmente mas barato y personal que un hotel, aunque 
sin sus comodidades. La palabra se utiliza principalmente en medios rurales. 
'''))